In [2]:
import sys
import os
sys.path.append("..")
from knool.remote_sensing.satellite import TLEsPerSat
from knool.geodata_processor import geo_info,geo_geom,geo_io
import pandas as pd
from skyfield.api import load
from datetime import datetime as dt
from datetime import timedelta
from skyfield.api import utc
import numpy as np
from osgeo import gdal, osr, ogr
import importlib

In [206]:
#HOW_TO_USE_SKYFIELD
##READ https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

tles = load.tle_file("../knool/remote_sensing/data/two_line_elements/sat40697.txt") # TLE読み込み
satellite=tles[0]
sgp4_model = satellite.model

print("衛星名: ", satellite.name)
print("元期: ", satellite.epoch.utc_jpl())
print("衛星番号: ", sgp4_model.satnum)
print("エポック年: ", sgp4_model.epochyr)
print("エポック日: ", sgp4_model.epochdays)
print("エポックのユリウス日: ", sgp4_model.jdsatepoch)
print("ndot: ", sgp4_model.ndot)
print("nddot: ", sgp4_model.nddot)
print("弾道抗力係数 B*: ", sgp4_model.bstar)
print("軌道傾斜角[rad]: ", sgp4_model.inclo)
print("昇交点赤経[rad]: ", sgp4_model.nodeo)
print("離心率: ", sgp4_model.ecco)
print("近地点引数[rad]: ", sgp4_model.argpo)
print("平均近点角[rad]: ", sgp4_model.mo)
print("平均運動[rad/min]: ", sgp4_model.no_kozai)

衛星名:  None
元期:  A.D. 2015-Jun-23 03:50:23.3849 UTC
衛星番号:  40697
エポック年:  15
エポック日:  174.15999288
エポックのユリウス日:  2457196.5
ndot:  -1.333237623051224e-12
nddot:  0.0
弾道抗力係数 B*:  0.0
軌道傾斜角[rad]:  1.7206276070908537
昇交点赤経[rad]:  4.337586504835915
離心率:  7.98e-05
近地点引数[rad]:  4.953066393576204
平均近点角[rad]:  2.5043519997356394
平均運動[rad/min]:  0.06244205289471824


In [207]:
#HOW_TO_USE_SKYFIELD
##READ https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

#get lat,lon,elevation from a tle
ts = load.timescale()  # skyfiled上で時刻を扱うモジュール
tle=tles[0]
date1 = dt(2018, 2, 1, 12, 15, 30, 2000, tzinfo=utc)
date1_sf=ts.from_datetime(date1)

geocentric=tle.at(date1_sf) #get xyz
subpoint = geocentric.subpoint()
lat = subpoint.latitude.degrees
lon = subpoint.longitude.degrees
ele = subpoint.elevation.m
print(geocentric.velocity)
print(geocentric.position)
print(geocentric.xyz)
print(lat,lon,ele)

[-0.00141461  0.00399833  0.00070632] au/day
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
-77.27650275778583 -162.68188667619526 815689.5183005631


In [208]:
#convert lla/ecef from ecef/lla using our package.
print(lat,lon,ele)
x,y,z=geo_info.convert_lla_to_ecef(lat,lon,ele)
print(x,y,z) # meter
lat,lon,ele=geo_info.convert_ecef_to_lla(x,y,z)
print(lat,lon,ele)

-77.27650275778583 -162.68188667619526 815689.5183005631
-1516878.3505331452 -472981.10875354183 -6995304.199162436
-77.27650276073706 -162.68188667619526 815689.5199458357


In [217]:
#create a kml file of a orbit line
infile=r"../knool/remote_sensing/data/two_line_elements/sat27424.txt"
test=TLEsPerSat(infile)

sdate=dt.strptime("2011-06-01 00:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-07-02 11:40:00",'%Y-%m-%d %H:%M:%S')
test.set_period(sdate, edate)

sdate=dt.strptime("2011-07-01 10:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-07-01 11:40:00",'%Y-%m-%d %H:%M:%S')
test.calc_positions_between(sdate,edate,1)
test.export("../../test_data/output/test.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:41: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(date), method='nearest')]]


In [218]:
#create a kml file of a orbit line
test.calc_positions_faster_between(sdate,edate,1)
test.export("../../test_data/output/test2.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:67: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [219]:
#Creates a kml file of a polygon
test.calc_buff_area_between(sdate,edate,1,1000000,ori="right")
test.export("../../test_data/output/test3.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:107: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [220]:
sdate=dt.strptime("2011-07-01 10:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-07-01 11:40:00",'%Y-%m-%d %H:%M:%S')
test.calc_scene_areas(sdate,edate,1,1450000,6,ori="middle")
test.export("../../test_data/output/test4.kml")

In [43]:
from knool.remote_sensing import satellite
importlib.reload(geo_info)
importlib.reload(geo_geom)
importlib.reload(geo_io)
importlib.reload(satellite)


shpfile=r"../../test_data/matchup/AOI2.shp"
vector=ogr.Open(shpfile)
layer=vector.GetLayer()
geom_list=[]
for feat in layer:
    geom_list.append(feat.GetGeometryRef().Clone())

trans_for=[]
trans_back=[]
for geom in geom_list:
    lon,lat,_=geom.Centroid().GetPoint()
    srs_pre=geo_info.get_shifted_wgs84_proj4(0)
    srs_af=geo_info.get_orthographic_proj4(lat,lon,0,0)
    trans_for.append(geo_info.get_coord_transform_proj4(srs_pre,srs_af))
    trans_back.append(geo_info.get_coord_transform_proj4(srs_af,srs_pre))

sdate=dt.strptime("2011-06-01 03:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-06-02 05:00:00",'%Y-%m-%d %H:%M:%S')

infile=r"../knool/remote_sensing/data/two_line_elements/sat38337.txt"
test=satellite.TLEsPerSat(infile)


In [47]:
# shpfile=r"../../test_data/matchup/AOI.shp"
# vector=ogr.Open(shpfile)
# layer=vector.GetLayer()
# geom_list=[]
# for feat in layer:
#     geom_list.append(feat.GetGeometryRef().Clone())

# trans_list=[]
# for geom in geom_list:
#     lon,lat,_=geom.Centroid().GetPoint()
#     srs_pre=geo_info.get_shifted_wgs84_proj4(0)
#     srs_af=geo_info.get_orthographic_proj4(lat,lon,0,0)
#     trans_fore=geo_info.get_coord_transform_proj4(srs_pre,srs_af)
#     trans_back=geo_info.get_coord_transform_proj4(srs_af,srs_pre)
#     trans_list.append([trans_fore,trans_back])

sdate=dt.strptime("2011-06-01 03:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-06-02 05:00:00",'%Y-%m-%d %H:%M:%S')
test.calc_scene_areas(sdate,edate,1,2330000,2,ori="middle")
test.export("../../test_data/output/test4.kml")

result,result2 = test.calc_intersect_scene_areas(sdate,edate,1,2330000,2,geom_list,trans_for,ori="middle")
result=test.transform(result,trans_back)
#print(len(result[0]),result)
test.export(r"../../test_data/output/test5.shp",option=0)
result=test.transform(result2,trans_back)
test.export(r"../../test_data/output/test6.kml",option=0)


In [53]:
sensor_name="amsr2"
sensor1=satellite.Sensor(sensor_name)
# sensor_name="modis"
# sensor2=satellite.Sensor(sensor_name)
print(sensor1.ncid)
sdate=dt.strptime("2011-06-01 03:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-06-01 04:00:00",'%Y-%m-%d %H:%M:%S')

geom_s1_llist00,_=sensor1.calc_intersect_scene_areas(sdate,edate,1,2,geom_list,trans_for,scene_all=True)
geom_s1_llist0,attr_s1_dlist=sensor1.union_adjacent_scenes(geom_s1_llist00)
geom_s1_llist=sensor1.transform(geom_s1_llist0,trans_back)
sensor1.export(r"../../test_data/output/test7.shp",option=0)
# sensor1.export(r"../../test_data/output/final2.shp",option=0)

38337


/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:108: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]
